<a href="https://colab.research.google.com/github/akitim/comptech-winter-school-2022_covid19-prediction/blob/main/predict_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# !pip install httpx
# !pip install prophet
# !mkdir tmp


In [47]:
from prophet import Prophet
import pandas as pd
import httpx
import numpy as np
import json
import pickle
from datetime import date, timedelta, datetime

def get_cases(days):
  data = httpx.get(f"https://storage.yandexcloud.net/covid-19/cases{\
      ('_' + str(days)) if days != 0 else ''}.pickle").content
  return pickle.loads(data)

# ================================PROPHET================================
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

from prophet import Prophet
import pandas as pd
import httpx
import numpy as np
import json


def get_train_data(countries, source_country, cases, field):
    df = pd.DataFrame(index=cases[cases['location'] == source_country]["date"],
                      columns=['y', 'ds'] + [i for i, _ in countries])
    df['y'] = cases[cases['location'] == source_country].reset_index().set_index("date")[field]
    df['ds'] = cases[cases['location'] == source_country].reset_index().set_index("date").index
    c = cases["location"].unique()

    for country, info in countries:
        if country in c:
            df[country] = cases[cases["location"] == country][field]
        else:
            del df[country]
    return df.fillna(method='ffill').dropna()


def get_df_future(days_predict, m, countries, cases, field, train_last):
    d = pd.date_range(train_last, periods=days_predict + 1, closed='right')
    future = pd.DataFrame()
    future["ds"] = d

    g = cases['location'].unique()

    for country, info in countries:
        if country in g:
            future[country] = cases[cases['location'] == country][field].iloc[
                              -days_predict + info["lag"]: info["lag"]].values

    return future.fillna(method='ffill')


def get_predict(days_predict, source_country, cases, lags) -> np.array:
    countries = sorted(filter(lambda x: -x[1]["lag"] > days_predict, lags[source_country].items()),
                       key=lambda x: x[1]["similarity"])[-2:]
    field = 'new_cases_smoothed'
    train = get_train_data(countries, source_country, cases, field)

    m = Prophet(
        daily_seasonality=False,
        yearly_seasonality=True,
        weekly_seasonality=True,
        changepoint_prior_scale=0.8,
        seasonality_mode='multiplicative'
    )
    for i in train.columns:
        if i != 'ds' and i != 'y':
            m.add_regressor(i, mode="multiplicative")
    m.fit(train)
    future = get_df_future(days_predict, m, countries, cases, field, train.index[-1])
    forecast = m.predict(future)

    return forecast[['ds', 'yhat']]
# =======================================================================
# =======================================================================

# =================================MEAN==================================
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
  
def get_predict_mean(country, n_days, need_predict, cases_all, use_filter=True, pred_from=None):

    field_name = ("filtered_" if use_filter else "") + "history"
    threshold = date(year=2021, month=1, day=1)

    just_cases = cases_all[country][field_name]
    if pred_from:
        mean_value = np.mean(just_cases[pred_from - n_days : pred_from].values)
    else:
        mean_value = np.mean(just_cases[- n_days : ].values)

    predict = np.asarray([mean_value for i in range(0, need_predict)])

    return predict

# =======================================================================
# =======================================================================

# ================================LINEAR=================================
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

# =======================================================================
# =======================================================================

# ================================HOLT=================================
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

# =======================================================================
# =======================================================================

# ================================SARIMA=================================
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

# =======================================================================
# =======================================================================

if __name__ == "__main__":
    with open("people_structure.csv", 'wb') as f:
        f.write(httpx.get("https://storage.yandexcloud.net/covid-19/people_structure.csv").content)

    df_sec = pd.read_csv("people_structure.csv")
    lags = json.loads(httpx.get("https://storage.yandexcloud.net/covid-19/lags.json").content)
    days_predict = 30

    index = None
    df = pd.DataFrame()
    for country in lags.keys():
        try:
            predict = get_predict(days_predict, country, df_sec, lags)
            if index is None:
                index = predict["ds"]

            d = pd.DataFrame()
            d["Date"] = index
            d["Country"] = country
            d["Prophet"] = predict["yhat"].values
            cases = get_cases(0)
            d["Mean_7"] = get_predict_mean(country, 7, 30, cases, use_filter=True)
            
            # Здесь добавляй в Linear, Mean+, Holt предикты

            #####

            df = pd.concat([df, d], ignore_index=True)
        except:
            continue

    df.to_csv("tmp/predict.csv")

In [48]:
data_ = pd.read_csv("tmp/predict.csv")
# data_["Mean"] = 0
data_[data_.Country == 'Honduras']
# data_.Country.unique()

,Unnamed: 0,Date,Country,Prophet,Mean
0,0,2022-02-04,Honduras,456.548368,315.607944
1,1,2022-02-05,Honduras,452.645061,315.607944
2,2,2022-02-06,Honduras,448.088367,315.607944
3,3,2022-02-07,Honduras,443.176556,315.607944
4,4,2022-02-08,Honduras,438.493282,315.607944
5,5,2022-02-09,Honduras,433.329024,315.607944
6,6,2022-02-10,Honduras,427.805901,315.607944
7,7,2022-02-11,Honduras,425.231192,315.607944
8,8,2022-02-12,Honduras,419.158911,315.607944
9,9,2022-02-13,Honduras,412.845689,315.607944


In [49]:
data_

,Unnamed: 0,Date,Country,Prophet,Mean
0,0,2022-02-04,Honduras,456.548368,315.607944
1,1,2022-02-05,Honduras,452.645061,315.607944
2,2,2022-02-06,Honduras,448.088367,315.607944
3,3,2022-02-07,Honduras,443.176556,315.607944
4,4,2022-02-08,Honduras,438.493282,315.607944
...,...,...,...,...,...
205,205,2022-03-01,Zimbabwe,343.676777,326.595061
206,206,2022-03-02,Zimbabwe,-7.278145,326.595061
207,207,2022-03-03,Zimbabwe,-386.119438,326.595061
208,208,2022-03-04,Zimbabwe,-787.990943,326.595061


In [50]:
get_cases(0)['Honduras']

{'filtered_history': Last_Update
 2020-03-22     22.385896
 2020-03-23     20.458503
 2020-03-24     18.692922
 2020-03-25     17.089153
 2020-03-26     15.647197
                  ...    
 2022-01-30    332.899018
 2022-01-31    317.894821
 2022-02-01    301.747186
 2022-02-02    284.456112
 2022-02-03    266.021600
 Length: 683, dtype: float64, 'filtered_validate': None, 'history': Last_Update
 2020-03-23     1.0
 2020-03-24     3.0
 2020-03-25     6.0
 2020-03-26    16.0
 2020-03-27    16.0
               ... 
 2022-01-31     0.0
 2022-02-01     0.0
 2022-02-02     0.0
 2022-02-03     0.0
 2022-02-04     0.0
 Name: Confirmed, Length: 683, dtype: float64, 'validate': None}

In [51]:
def hwes_predict(data, country, days):
    # import library
    from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES
  
    # create time series for country
    df = data[data['location'] == country]
    ts = df.set_index(pd.DatetimeIndex(df['date'])).asfreq('D').total_cases
  
    # split the time series
    train = ts.iloc[:-days]

    # fit the model
    model = HWES(train, seasonal_periods=4, trend='mul', seasonal='add')
    fitted = model.fit()

    predict = fitted.forecast(days)

  return predict

array([326.59506127, 326.59506127, 326.59506127, 326.59506127,
       326.59506127, 326.59506127, 326.59506127, 326.59506127,
       326.59506127, 326.59506127, 326.59506127, 326.59506127,
       326.59506127, 326.59506127, 326.59506127, 326.59506127,
       326.59506127, 326.59506127, 326.59506127, 326.59506127,
       326.59506127, 326.59506127, 326.59506127, 326.59506127,
       326.59506127, 326.59506127, 326.59506127, 326.59506127,
       326.59506127, 326.59506127])